In [194]:
# Import required libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [195]:
# Import the csv file: Accident_Data
df = pd.read_csv('C:\\Users\Servet\Desktop\Project1 Files\Accidents.csv')
df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201,3,2/8/2016 5:46,2/8/2016 11:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201,2,2/8/2016 6:07,2/8/2016 6:37,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201,2,2/8/2016 6:49,2/8/2016 7:19,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201,3,2/8/2016 7:23,2/8/2016 7:53,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201,2,2/8/2016 7:39,2/8/2016 8:09,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [196]:
# Get some summary details about the Data Frame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 49 columns):
ID                       1048575 non-null object
Source                   1048575 non-null object
TMC                      1048575 non-null int64
Severity                 1048575 non-null int64
Start_Time               1048575 non-null object
End_Time                 1048575 non-null object
Start_Lat                1048575 non-null float64
Start_Lng                1048575 non-null float64
End_Lat                  0 non-null float64
End_Lng                  0 non-null float64
Distance(mi)             1048575 non-null float64
Description              1048574 non-null object
Number                   396803 non-null float64
Street                   1048575 non-null object
Side                     1048575 non-null object
City                     1048548 non-null object
County                   1048575 non-null object
State                    1048575 non-null object
Zipcode       

We observe that, some columns have missing values and two columns have no values. We will firstly focus on filling the missing values.

In [197]:
#Since there are no informatiojn in two columns, we will drop them
df = df.drop(columns = ['End_Lat', 'End_Lng'])
print('The DataFrame has {} rows and {} columns'.format(df.shape[0],df.shape[1]))

The DataFrame has 1048575 rows and 47 columns


In [198]:
# Let's check the total missing values in each entry
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values_list = missing_values[missing_values>=1]
missing_values_list

Precipitation(in)        927055
Wind_Chill(F)            875145
Number                   651772
Wind_Speed(mph)          207628
Visibility(mi)            28973
Weather_Condition         28951
Humidity(%)               25111
Temperature(F)            24059
Pressure(in)              21069
Wind_Direction            17639
Weather_Timestamp         17631
Airport_Code              10290
Timezone                    581
Zipcode                     140
Nautical_Twilight            29
Astronomical_Twilight        29
Civil_Twilight               29
Sunrise_Sunset               29
City                         27
Description                   1
dtype: int64

In [199]:
# When and humidity, precipitation and wind speed values are missing, it is more likely to have 0 precipitation
df['Precipitation(in)']=df['Precipitation(in)'].fillna(0)
df['Humidity(%)'] = df['Humidity(%)'].fillna(0)
df['Wind_Speed(mph)'] = df['Wind_Speed(mph)'].fillna(0)

In [200]:
# Pressure(in)  column can be filled with average values
df['Pressure(in)'] = df['Pressure(in)'].fillna(df['Pressure(in)'].mean())

In [201]:
# Temperature(F)  column can be filled with average Temperature for that state
df['Temperature(F)'] = df.groupby('State')['Temperature(F)'].transform(lambda grp: grp.fillna(grp.mean()))


In [202]:
# We will fill the 'Wind_Chill(F)' values by corresponding Temperatures.
df['Wind_Chill(F)'] = df['Wind_Chill(F)'].fillna(df['Temperature(F)'])

In [203]:
# Let's check the total missing values once again
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values_list = missing_values[missing_values>=1]
missing_values_list

Number                   651772
Visibility(mi)            28973
Weather_Condition         28951
Wind_Direction            17639
Weather_Timestamp         17631
Airport_Code              10290
Timezone                    581
Zipcode                     140
Astronomical_Twilight        29
Nautical_Twilight            29
Civil_Twilight               29
Sunrise_Sunset               29
City                         27
Description                   1
dtype: int64

In [204]:
# We will fill the Weather Condition and Description
df['Weather_Condition'] = df['Weather_Condition'].fillna('NaN')
df['Description'] = df['Description'].fillna('Accident')

In [205]:
# Let's check the total missing values once again
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values_list = missing_values[missing_values>=1]
missing_values_list

Number                   651772
Visibility(mi)            28973
Wind_Direction            17639
Weather_Timestamp         17631
Airport_Code              10290
Timezone                    581
Zipcode                     140
Astronomical_Twilight        29
Nautical_Twilight            29
Civil_Twilight               29
Sunrise_Sunset               29
City                         27
dtype: int64

In [206]:
# We will fill the missing values in "city" column by the most coommon cities in that city 
df['City'] = df.groupby('State')['City'].transform(lambda grp: grp.fillna(grp.value_counts().index[0]))

In [207]:
# We will fill the 'Timezone' column by the most coommon time zones in that  State
df['Timezone'] = df.groupby('State')['Timezone'].transform(lambda tz: tz.fillna(tz.value_counts().index[0]))

In [208]:
# We will fill the "Zipcode" and "Airport_Code" the most coommon time zones in that State
df['Zipcode'] = df.groupby('State')['Zipcode'].transform(lambda zc: zc.fillna(zc.value_counts().index[0]))
df['Airport_Code'] = df.groupby('State')['Airport_Code'].transform(lambda ac: ac.fillna(ac.value_counts().index[0]))

In [209]:
# Let's check the total missing values once again
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values_list = missing_values[missing_values>=1]
missing_values_list

Number                   651772
Visibility(mi)            28973
Wind_Direction            17639
Weather_Timestamp         17631
Nautical_Twilight            29
Astronomical_Twilight        29
Civil_Twilight               29
Sunrise_Sunset               29
dtype: int64

In [210]:
missing_df = pd.concat([missing_values,df[missing_values.index].dtypes], axis=1, keys=['Total','Data Types'],sort=False)
missing_df[missing_df['Total']>=1]

,Total,Data Types
Number,651772,float64
Visibility(mi),28973,float64
Wind_Direction,17639,object
Weather_Timestamp,17631,object
Nautical_Twilight,29,object
Astronomical_Twilight,29,object
Civil_Twilight,29,object
Sunrise_Sunset,29,object


In [211]:
missing_copy = missing_df[missing_df['Total']>=1].copy()

In [212]:
object_columns = missing_copy[missing_copy['Data Types']=='object'].index
df[object_columns].head()

,Wind_Direction,Weather_Timestamp,Nautical_Twilight,Astronomical_Twilight,Civil_Twilight,Sunrise_Sunset
0,Calm,2/8/2016 5:58,Night,Night,Night,Night
1,Calm,2/8/2016 5:51,Night,Day,Night,Night
2,SW,2/8/2016 6:56,Day,Day,Night,Night
3,SW,2/8/2016 7:38,Day,Day,Day,Night
4,SW,2/8/2016 7:53,Day,Day,Day,Day


In [213]:
# Let's check the total missing values once again
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values_list = missing_values[missing_values>=1]
missing_values_list

Number                   651772
Visibility(mi)            28973
Wind_Direction            17639
Weather_Timestamp         17631
Nautical_Twilight            29
Astronomical_Twilight        29
Civil_Twilight               29
Sunrise_Sunset               29
dtype: int64

In [214]:
# Convert Start_Time and End_Time to datetypes
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')
df['End_Time'] = pd.to_datetime(df['End_Time'], errors='coerce')

# Extract year, month, day, hour and weekday
df['Year']=df['Start_Time'].dt.year
df['Month']=df['Start_Time'].dt.strftime('%b')
df['Day']=df['Start_Time'].dt.day
df['Hour']=df['Start_Time'].dt.hour
df['Weekday']=df['Start_Time'].dt.strftime('%a')

In [215]:
# We will fill "Day' and 'Night' according to the Start Time. We will define a function: Filler and pass those columns into the function
def filler(df,columns):
    # get list comprising column missing data
    lst = df[df[columns].isna()].index
    for i in lst:
        if 6<= df.loc[i,'Start_Time'].hour and df.loc[i,'Start_Time'].hour <18:
            df[columns] = df[columns].fillna('Day')
        else:
            df[columns] = df[columns].fillna('Night')


filler(df,'Nautical_Twilight')
filler(df,'Sunrise_Sunset')
filler(df,'Civil_Twilight')
filler(df,'Astronomical_Twilight')

In [216]:
# We will fill Weather Timestamp with Start_Time
df['Weather_Timestamp'] = df['Weather_Timestamp'].fillna(df['Start_Time'])

In [223]:
# We will fill the missing Wind Dircetions with the most common values in corresponding 
df['Wind_Direction'] = df.groupby('City')['Wind_Direction'].transform(lambda wnd: wnd.fillna(wnd.value_counts().max()))

In [224]:
# Let's check the total missing values once again
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values_list = missing_values[missing_values>=1]
missing_values_list

Number            651772
Visibility(mi)     28973
Wind_Direction      6500
dtype: int64

In [227]:
# Still we will fill the missing Wind Dircetions with the most common values in corresponding state
df['Wind_Direction'] = df.groupby('State')['Wind_Direction'].transform(lambda wnd: wnd.fillna(wnd.value_counts().max()))

In [228]:
# Since Visibility(mi) is float64, we can fill with mean
df['Visibility(mi)'] = df['Visibility(mi)'].fillna(df['Visibility(mi)'].mean())

In [229]:
# Let's check the total missing values once again
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values_list = missing_values[missing_values>=1]
missing_values_list

Number    651772
dtype: int64

In [231]:
#We fill each missing number by the average number on each zipcode
df['Number'] = df.groupby('Zipcode')['Number'].transform(lambda nmbr: nmbr.fillna(nmbr.mean()))

In [232]:
# Let's check the total missing values once again
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values_list = missing_values[missing_values>=1]
missing_values_list

Number    87768
dtype: int64

There are still some missing values, due to the no street number recorded in some zipcodes. So we can use the same grouping process with average number on state.

In [233]:
df['Number'] = df.groupby('State')['Number'].transform(lambda nmbr: nmbr.fillna(nmbr.mean()))

In [234]:
# Let's check the total missing values once again
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values_list = missing_values[missing_values>=1]
missing_values_list

Number    5
dtype: int64

In [235]:
# Now only 5 numbers are left, and we will fill with those 5 with mean
df['Number']  = df['Number'] .fillna(df['Number'] .mean())

In [236]:
# Let's check the total missing values once again
missing_values = df.isnull().sum().sort_values(ascending=False)
missing_values_list = missing_values[missing_values>=1]
missing_values_list

Series([], dtype: int64)

Finally, we fill all the missing values, and now we can get some information about the Data Frame

In [237]:
df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,...,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Year,Month,Day,Hour,Weekday
0,A-1,MapQuest,201,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,...,False,Night,Night,Night,Night,2016,Feb,8,5,Mon
1,A-2,MapQuest,201,2,2016-02-08 06:07:00,2016-02-08 06:37:00,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,...,False,Night,Night,Night,Day,2016,Feb,8,6,Mon
2,A-3,MapQuest,201,2,2016-02-08 06:49:00,2016-02-08 07:19:00,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,...,False,Night,Night,Day,Day,2016,Feb,8,6,Mon
3,A-4,MapQuest,201,3,2016-02-08 07:23:00,2016-02-08 07:53:00,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,...,False,Night,Day,Day,Day,2016,Feb,8,7,Mon
4,A-5,MapQuest,201,2,2016-02-08 07:39:00,2016-02-08 08:09:00,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,...,False,Day,Day,Day,Day,2016,Feb,8,7,Mon


In [238]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 52 columns):
ID                       1048575 non-null object
Source                   1048575 non-null object
TMC                      1048575 non-null int64
Severity                 1048575 non-null int64
Start_Time               1048575 non-null datetime64[ns]
End_Time                 1048575 non-null datetime64[ns]
Start_Lat                1048575 non-null float64
Start_Lng                1048575 non-null float64
Distance(mi)             1048575 non-null float64
Description              1048575 non-null object
Number                   1048575 non-null float64
Street                   1048575 non-null object
Side                     1048575 non-null object
City                     1048575 non-null object
County                   1048575 non-null object
State                    1048575 non-null object
Zipcode                  1048575 non-null object
Country                  1048575 non

In [255]:
# Now, we will convert the first cleaned version to a csv:
df.to_csv('US_Accidents_cleaned1.csv')

In [249]:
# We will check some of the objects that will make a category
df.Weather_Condition.value_counts()

Clear                                  386111
Overcast                               182238
Mostly Cloudy                          142518
Partly Cloudy                           95093
Scattered Clouds                        94684
Light Rain                              50388
NaN                                     28951
Haze                                    14561
Light Snow                              13948
Rain                                    11829
Fog                                      5320
Heavy Rain                               4377
Light Drizzle                            3853
Light Thunderstorms and Rain             2356
Thunderstorm                             2125
Snow                                     1308
Smoke                                    1262
Heavy Thunderstorms and Rain             1108
Thunderstorms and Rain                   1020
Mist                                      924
Patches of Fog                            840
Light Freezing Rain               

In [250]:
# This can be a value that we can use dummy variables to categorize each condition
df = pd.get_dummies(df, columns=['Weather_Condition'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Columns: 107 entries, ID to Weather_Condition_Widespread Dust
dtypes: bool(13), datetime64[ns](2), float64(11), int64(5), object(20), uint8(56)
memory usage: 373.0+ MB


In [251]:
# Now, we can find the descriptions of the numerical values
df.describe()

,TMC,Severity,Start_Lat,Start_Lng,Distance(mi),Number,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),...,Weather_Condition_Small Hail,Weather_Condition_Smoke,Weather_Condition_Snow,Weather_Condition_Snow Grains,Weather_Condition_Snow Showers,Weather_Condition_Squalls,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Volcanic Ash,Weather_Condition_Widespread Dust
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,...,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,2.077632e+02,2.358323e+00,3.611783e+01,-9.517610e+01,1.476137e-01,5.267198e+03,6.199550e+01,6.082627e+01,6.520202e+01,3.003558e+01,...,1.621248e-05,1.203538e-03,1.247407e-03,9.536752e-07,9.536752e-07,1.239778e-05,2.026560e-03,9.727487e-04,1.621248e-05,4.673009e-05
std,1.968127e+01,4.824762e-01,4.895696e+00,1.720663e+01,1.361161e+00,7.388160e+03,1.878450e+01,2.091214e+01,2.385668e+01,2.188615e-01,...,4.026442e-03,3.467118e-02,3.529663e-02,9.765630e-04,9.765630e-04,3.521028e-03,4.497171e-02,3.117376e-02,4.026442e-03,6.835784e-03
min,2.000000e+02,0.000000e+00,2.457027e+01,-1.244931e+02,0.000000e+00,1.000000e+00,-7.780000e+01,-7.780000e+01,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.010000e+02,2.000000e+00,3.300991e+01,-1.173272e+02,0.000000e+00,1.217526e+03,5.000000e+01,5.000000e+01,4.900000e+01,2.992000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.010000e+02,2.000000e+00,3.541019e+01,-9.021603e+01,0.000000e+00,3.269000e+03,6.400000e+01,6.400000e+01,6.800000e+01,3.003000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.010000e+02,3.000000e+00,4.009613e+01,-8.093151e+01,1.000000e-02,7.103000e+03,7.590000e+01,7.590000e+01,8.500000e+01,3.014000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,4.060000e+02,4.000000e+00,4.900076e+01,-6.865836e+01,3.336300e+02,8.519090e+05,1.670000e+02,1.670000e+02,1.000000e+02,3.304000e+01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


We want to perform our first linear model to see the strenght of our model

In [245]:
from sklearn.linear_model import LinearRegression

In [246]:
lr = LinearRegression()
lr

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [252]:
# Now, we will define target and feature variables. Since this project is trying to generate a model to predict the severity of accidents, we will use "Severrity" as targetr variable
target_names = ['Severity']
feature_names = [c for c in df.columns if c not in target_names and df[c].dtype != object and not c.endswith('Time')]

In [253]:
X = df[feature_names]
y = df[target_names]
X.shape

(1048575, 84)

In [254]:
lr.fit(X, y)
lr.score(X,y)

0.12356542042762264

Nice try, now I will save this DataFrame with dummies to a csv

In [256]:
df.to_csv('US_Accidents_with_dummies.csv')